In [1]:
#_____Main_bot________


# INTEGRANTES:



## 1- STEVEN AMPIE
## 2- ENRIQUE QUERINI
## 3- VIVIANA DAVIS
## 4- JOHAN PIMENTEL
## 5- ALEXANDER SANTAMARIA

In [3]:
import FuncionRequisicion as rq
import tensorflow 
import threading
import telebot
import numpy as np
import tflearn
import nltk
import json
import pickle
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()
import random
from telebot.types import ReplyKeyboardMarkup
from telebot.types import ForceReply
from Token import *

ModuleNotFoundError: No module named 'Token'

In [ ]:
rq.crearCotizacion('rigoberto', 'caja de mascarilla', 2, 'nosebro', 'WN', 2, 5)

In [ ]:
with open("Frases.json", encoding='utf-8') as files:
    frases=json.load(files)

try:
    with open("Variables.pickle","rb") as archivoPickle_nuevo:
        palabras_a_usar, etiquetas, train, output = pickle.load(archivoPickle_nuevo)  

except:
    palabras_a_usar=[]
    etiquetas=[]
    auxX=[]
    auxY=[]


    nltk.download('punkt')



    for content in frases["contenido"]:
        for patrones in content["patrones"]:
                auxPalabra=nltk.word_tokenize(patrones)
                palabras_a_usar.extend(auxPalabra)
                auxX.append(auxPalabra)
                auxY.append(content["tag"])
                if content["tag"] not in etiquetas:
                    etiquetas.append(content["tag"])


    palabras_a_usar=[stemmer.stem(w.lower()) for w in palabras_a_usar if w!= "?"]

    palabras_a_usar=sorted(list(set(palabras_a_usar)))
    etiquetas=sorted(etiquetas)

    train=[]
    output=[]
    salidaVacia=[0 for _ in range(len(etiquetas))]




    for x, documento in enumerate(auxX):
            cubeta=[]
            auxPalabra=[stemmer.stem(w.lower()) for w in documento]
            for w in palabras_a_usar:
                if w in auxPalabra:
                    cubeta.append(1)
                else:
                    cubeta.append(0)
            filasalida=salidaVacia[:]
            filasalida[etiquetas.index(auxY[x])]=1
            train.append(cubeta)
            output.append(filasalida)
            


    train=np.array(train)
    output=np.array(output)
    with open("variables_nuevas.pickle", "wb") as archivoPickle:
        pickle.dump((palabras_a_usar, etiquetas,train, output ), archivoPickle )

In [ ]:
red=tflearn.input_data(shape=[None,len(train[0])])
red=tflearn.fully_connected(red, len(output))
red=tflearn.fully_connected(red,len(output))
red=tflearn.fully_connected(red,len(output[0]), activation="softmax") 
red=tflearn.regression(red)

modelo=tflearn.DNN(red)
try:
     modelo.load("modelo_bot.tflearn")

except:
    
     modelo.fit(train, output, n_epoch=1000, batch_size=len(output), show_metric=True)
     modelo.save("modelo_bot.tflearn")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Restoring parameters from c:\Users\Steven\Desktop\Tech_Lighters\Tech-Lighters\modelo_bot.tflearn


In [ ]:

doc=open("Cotizacion.docx", "rb")

In [ ]:

bot=telebot.TeleBot(Token) #utilizamos el token del bot

#funciones implementadas en el bot usando la api de telegram





# {  Funcion reservada para administradores o individuos pertenecientes a la empresa.


@bot.message_handler(commands=["admin"])
def cmd_admin(message):
     
     markup=ForceReply()
     mensaje=bot.send_message(message.chat.id, "Ingrese su nombre de administrador", reply_markup=markup)
     
     bot.register_next_step_handler(mensaje, is_admin)

def is_admin(message): #Funcion reservada para admins 
     
    with open("Usuarios.json", "r+") as File:
            File=json.load(File)

    if [message.text] in File["usuarios"]["Users"]:
          bot.send_message(message.chat.id, "Usuario encontrado")
    else:
        bot.send_message(message.chat.id, "Usuario no encontrado")

#                                          } 









#{          COMANDOS DE AYUDA

@bot.message_handler(commands=["start", "help", "ayuda"])
def cmd_start(message):
     bot.send_message(message.chat.id, "Hola que tal, soy Tommy su bot-asistente, en que le puedo ayudar? ")





#                                  }




# {

@bot.message_handler(commands=["cotizacion"])
def cmd_cotizacion(message):
     markup=ForceReply()
     name_lastname=bot.send_message(message.chat.id, "Ingrese su nombre y apellido", reply_markup=markup)
     bot.register_next_step_handler(name_lastname,siguiente)



prueba=''
def siguiente(message):
     randome=message.text
     markup=ForceReply()
     bot.send_message(message.chat.id, "Excelente!, ahora")
     producto_name=bot.send_message(message.chat.id, "ingrese el nombre de su producto", reply_markup=markup)
     bot.register_next_step_handler(producto_name,cantidad)
    
     


def cantidad(message):

     producto_name=message.text
     markup=ForceReply()
     cant_productos=bot.send_message(message.chat.id, "ingrese la cantidad de productos que desea", reply_markup=markup)
     bot.register_next_step_handler(cant_productos, create_cotizacion )



def create_cotizacion(message):
       
       global randome
       global producto_name
       global prueba
       cant=message.text
       rq.crearCotizacion(prueba,"caja de mascarilla" , 2, 'nosebro', 'WN', 2, 5)
       salida=bot.send_message(message.chat.id, "Ingresa recibido, obtener la cotizacion")
       bot.register_next_step_handler(salida, send_document)


    
def send_document(message):
     markup=ForceReply()
     global doc
    # bot.send_message(message.chat.id, "")
     bot.send_document(message.chat.id, doc)




                                  # }







#{ Funcion de procesamiento de lenguaje natural y respuesta acorde a lo entrenado

@bot.message_handler(content_types=["text"])
def greetings(message):

        msg=message.text

        cubeta=[0 for _ in range(len(palabras_a_usar))]
        enterProcessed=nltk.word_tokenize(msg)
        enterProcessed=[stemmer.stem(word.lower()) for word in enterProcessed]
        for singleWord in enterProcessed:
            for i, word in enumerate(palabras_a_usar):
                if word == singleWord:
                    cubeta[i]=1
        resultados=modelo.predict([np.array(cubeta)])
        
        resultadosIndices=np.argmax(resultados)
        tag=etiquetas[resultadosIndices]


        for tagAux in frases["contenido"]:
            if tagAux["tag"] == tag:
                    respuesta=tagAux["respuestas"]
                    respuesta_real=random.choice(respuesta)
        if max(cubeta) == 0:
            bot.send_message(message.chat.id,'No entiendo lo que dices')
            bot.send_message(message.chat.id, "Para conocer nuestros comandos, ingresa /help ")
        else:
            bot.send_message(message.chat.id,respuesta_real)
            bot.send_message(message.chat.id, "En que te puedo ayudar?")


#                                   }












     



In [ ]:
#mantener el bot activo
#def iniciar_bot():
 #   bot.infinity_polling()

In [ ]:

if __name__ == '__main__':
    print('iniciando el bot')
    bot.infinity_polling()
    #inicio=threading.Thread(name="inicio", target=iniciar_bot)
#    inicio.start()
    print('fin')    

iniciando el bot


2023-01-24 15:54:09,985 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2023-01-24 15:54:09,987 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"


fin
